# This scraper returns user comments to posts, where the searched company is tagged on Linkedin
# Useful for a sentiment analysis

In [ ]:
import pandas as pd
import numpy as np

from getpass import getpass
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

## Login and Company Search
#### Put a Linkedin username / password and write the company brand

In [ ]:
username = input("USERNAME: ")
password = getpass("PASSWORD: ")
azienda = input("COMPANY: ")

driver = webdriver.Chrome()
driver.get('https://www.linkedin.com')
username_field = driver.find_element(By.ID, 'session_key')
password_field = driver.find_element(By.ID, 'session_password')
username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.ENTER)

# Go to company Linkedin page
search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="global-nav-typeahead"]/input')))
search_bar.send_keys(azienda)
search_bar.send_keys(Keys.ENTER)

# Go to Post section
time.sleep(5)
posts_section = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')))
posts_section.send_keys(Keys.ENTER)

## Scraper

In [ ]:
def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
        
def click_load_more_comments():
    load_more_comments_buttons = driver.find_elements(By.XPATH, "//span[@class='artdeco-button__text' and text()='Carica altri commenti']")
    for button in load_more_comments_buttons:
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)

def get_comments():
    new_comments = driver.find_elements(By.XPATH, "//span[@dir='ltr']")
    updated_comments = []
    for comment in new_comments:
        updated_comments.append(comment.text)
    return updated_comments


all_comments = []

start_time = time.time()
total_time = 400             # Linkedin post page is dynamic, so we need to scroll until enough comments are loaded (for example 7,5 min of scroll = 400 seconds )

while True:
    scroll_to_bottom()
    elapsed_time = time.time() - start_time
    if elapsed_time >= total_time:
        break
click_load_more_comments()
time.sleep(60)
get_comments()

all_comments = get_comments()

In [ ]:
comments_raw=pd.DataFrame(all_comments)
comments=comments_raw.copy()
comments.info()